In [1]:
from tensorflow.keras.models import model_from_json
file_prefix = "new_jester_3DCNN"
def load_model(file_path):
    # Load the model architecture from the JSON file
    with open(file_path + '.json', "r") as json_file:
        model_json = json_file.read()
    
    model = model_from_json(model_json)
    
    # Load weights into the model
    model.load_weights(file_path + '.h5')
    
    return model

# Usage
loaded_model = load_model('{}_model'.format(file_prefix))

# Compile the model if needed (especially if you want to evaluate or predict)
# loaded_model.compile(optimizer='SGD(learning_rate=0.001)', 
#                      loss='categorical_crossentropy', 
#                      metrics=['accuracy'])

# You can now use loaded_model for evaluation or predictions


2024-10-17 16:16:53.921989: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-17 16:16:53.947536: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-17 16:16:54.372719: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce GTX 1660 Ti, compute capability 7.5


2024-10-17 16:16:54.934910: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-10-17 16:16:54.949933: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-10-17 16:16:54.950049: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [2]:
loaded_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d (Conv3D)             (None, 34, 30, 30, 64)    5248      
                                                                 
 batch_normalization (BatchN  (None, 34, 30, 30, 64)   256       
 ormalization)                                                   
                                                                 
 max_pooling3d (MaxPooling3D  (None, 34, 15, 15, 64)   0         
 )                                                               
                                                                 
 conv3d_1 (Conv3D)           (None, 32, 13, 13, 128)   221312    
                                                                 
 batch_normalization_1 (Batc  (None, 32, 13, 13, 128)  512       
 hNormalization)                                                 
                                                        

In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.layers import Conv3D, BatchNormalization, MaxPool3D, Dense, Flatten, Dropout, ConvLSTM2D, LSTM
from tensorflow.keras.models import model_from_json

##### importing libraries

In [4]:
import pandas as pd
import numpy as np
from glob import glob
import skimage.transform
from skimage import io
from sklearn.preprocessing import OneHotEncoder
import joblib

import tensorflow as tf

#### Selecting Gestures

In [5]:
gesture_list = ['Swiping Right','Swiping Left','Thumb Up','Thumb Down','No gesture','Zooming In With Full Hand','Zooming Out With Full Hand']

#train
file_prefix = "new_jester_3DCNN_lstm"
df = pd.read_csv('./annotations/jester-v1-train.csv',header=None,names=['id','labels'])
df = df[df['labels'].isin(gesture_list)]
df.to_csv('{}_train.csv'.format(file_prefix),sep=';',index=False)
# pd.read_csv('jester-v1-8classes_train.csv',sep=";").head()
print(df.shape)
print(df.head())
#val
df = pd.read_csv('./annotations/jester-v1-validation.csv',sep=';',header=None,names=['id','labels'])
df = df[df['labels'].isin(gesture_list)]
df.to_csv('{}_val.csv'.format(file_prefix),sep=';',index=False)
print(df.shape)
print(df.head())

print('Csvs done')

class DataGenerator(tf.keras.utils.Sequence):
    def __init__(self, file_path, batch_size=2, image_dim=(256,256), frames_count=36, n_channels=1, base_dir='./20bn-jester-v1/', n_classes=27,validation=False):
        self.image_dim = image_dim
        self.batch_size = batch_size
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = True 
        self.frames_count = frames_count
        self.df = pd.read_csv(file_path,sep=";")
        self.df.id = self.df.id.map(str)
        if "train" in file_path:
            self.encoder = OneHotEncoder(sparse_output=False)
            self.encoder.fit(self.df.labels.values[:,None])
            joblib.dump(self.encoder,"{}_encoder_joblib.joblib".format('_'.join(file_path.split('_')[:-1])))
            np.save("encoder_classes_{}_npy.npy".format(n_classes),self.encoder.categories_)
        else:
            self.encoder = joblib.load("{}_encoder_joblib.joblib".format('_'.join(file_path.split('_')[:-1])))
        self.base_dir = base_dir
        self.on_epoch_end()

    def __len__(self):
        ## Decides step_size
        return self.df.shape[0] // self.batch_size

    def __getitem__(self, index):
        'Generate one batch of data'
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        list_IDs_temp = self.df.loc[indexes,"id"].to_list()
        X, y = self.__data_generation(indexes)
        return X, y 

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(self.df.shape[0])
        np.random.shuffle(self.indexes)

    def __data_generation(self, indexes):
        X = np.empty((self.batch_size,self.frames_count, *self.image_dim, self.n_channels))
        y = np.empty((self.batch_size,1), dtype=str)
        y = []
        for i, ID in enumerate(indexes):
            files_list = self.standardize_frame_count(glob(self.base_dir + self.df.loc[ID,"id"] + "/*.jpg"),self.df.loc[ID])
            for idx,filename in enumerate(files_list):
                X[i,idx] = tf.keras.preprocessing.image.img_to_array(tf.keras.preprocessing.image.load_img(filename,color_mode='grayscale',target_size=self.image_dim))
            y.append(self.df.loc[ID,"labels"])
        encoded = self.encoder.transform(np.array(y)[:,None])
        return X,encoded
        
    def standardize_frame_count(self,files,error_check):
        shape = len(files)
        if shape < self.frames_count:
            to_add = self.frames_count - shape
            mid  = len(files)//2
            dup = [files[mid]]*to_add
            files = files[:mid] + dup + files[mid+1:]
        elif shape > self.frames_count:
            to_remove = (shape - self.frames_count)
            to_remove = int(to_remove) if int(to_remove) == to_remove else int(to_remove) + 1
            files = files[to_remove:]
        return files
    
params = {'batch_size': 56,
          'n_classes': len(gesture_list),
          'n_channels': 3,
          'image_dim': (32,32)
          }

# Generators
training_generator = DataGenerator(file_path="{}_train.csv".format(file_prefix),**params)
validation_generator = DataGenerator(file_path="{}_val.csv".format(file_prefix),**params)


(29819, 2)
        id                      labels
7   136859                    Thumb Up
8    68574               Swiping Right
13   20706                  No gesture
14   42237                  Thumb Down
19  133442  Zooming Out With Full Hand
(3640, 2)
       id                     labels
0    9223                   Thumb Up
2   42920               Swiping Left
3  106485                 Thumb Down
6   35341  Zooming In With Full Hand
7   94928              Swiping Right
Csvs done


In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.layers import Conv3D, BatchNormalization, MaxPool3D, Dense, Flatten, Dropout, ConvLSTM2D, LSTM
from tensorflow.keras.models import model_from_json
#from keras.optimizers import SGD
from tensorflow.keras.losses import CategoricalCrossentropy

In [7]:
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy('mixed_float16')

In [8]:
input_shape = (36, 32, 32, 3)
weight_decay = 0.00005


#### Creating custom loss function

In [9]:
def custom_categorical_crossentropy(y_true, y_pred):
    y_pred = tf.clip_by_value(y_pred, 1e-8, 1.0)  # Prevent log(0)
    return tf.keras.losses.categorical_crossentropy(y_true, y_pred)


In [10]:
import tensorflow as tf
from tensorflow.keras.layers import (Conv3D, MaxPooling3D, Dropout, Dense, BatchNormalization, Flatten,GlobalAveragePooling3D)
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import backend as K
from tensorflow.keras.regularizers import l2


In [ ]:
def build_model(n_classes=7):
    momentum = 0.99
    
    model = Sequential()
    model.add(Conv3D(36, kernel_size=3, strides=1, padding='same', activation='elu', input_shape=input_shape))
    model.add(BatchNormalization(momentum=momentum))
    model.add(MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2)))

    model.add(Conv3D(64, kernel_size=3, strides=1, padding='same', activation='elu'))
    model.add(BatchNormalization(momentum=momentum))
    model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2)))

    model.add(Conv3D(64, kernel_size=3, strides=1, padding='same', activation='elu'))
    model.add(BatchNormalization(momentum=momentum))
    model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2)))

    model.add(ConvLSTM2D(filters=64, kernel_size=(3, 3),
                         padding='same', return_sequences=True))
    model.add(BatchNormalization())
    
    model.add(ConvLSTM2D(filters=64, kernel_size=(3, 3),
                         padding='same', return_sequences=True))
    model.add(BatchNormalization())

    model.add(GlobalAveragePooling3D())
    model.add(Dropout(0.3))  # Reduced dropout rate
    model.add(Dense(n_classes, activation='softmax', kernel_regularizer=l2(0.01)))

    return model

model = build_model(n_classes=len(gesture_list))
optimizer = SGD(learning_rate=0.0001)
model.compile(
    optimizer=optimizer,
    loss=CategoricalCrossentropy(from_logits=True), 
    metrics=['accuracy']
)

# Define EarlyStopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model with EarlyStopping
h = model.fit(
    training_generator,
    validation_data=validation_generator,
    validation_steps=32,
    epochs=30,
    verbose=1,
    callbacks=[early_stopping]
)

# Save the model
def save_model(model, file_path):
    model_json = model.to_json()
    with open(file_path + '.json', "w") as json_file:
        json_file.write(model_json)
    model.save_weights(file_path + '.h5')

save_model(model, '{}_model'.format(file_prefix))


Epoch 1/30


2024-10-17 16:17:01.193703: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
/home/mca/anaconda3/envs/dse/lib/python3.8/site-packages/keras/backend.py:5561: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(
2024-10-17 16:17:03.950193: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8600


  1/532 [..............................] - ETA: 5:59:50 - loss: 2.1522 - accuracy: 0.1429

2024-10-17 16:17:41.701932: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x7f7d04005150 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-10-17 16:17:41.701958: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (0): NVIDIA GeForce GTX 1660 Ti, Compute Capability 7.5
2024-10-17 16:17:41.746004: I ./tensorflow/compiler/jit/device_compiler.h:180] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


532/532 [==============================] - ETA: 0s - loss: 2.0935 - accuracy: 0.1780

2024-10-17 16:20:07.385504: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
/home/mca/anaconda3/envs/dse/lib/python3.8/site-packages/keras/backend.py:5561: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


532/532 [==============================] - 195s 291ms/step - loss: 2.0935 - accuracy: 0.1780 - val_loss: 2.0205 - val_accuracy: 0.2333
Epoch 2/30
532/532 [==============================] - 155s 291ms/step - loss: 2.0230 - accuracy: 0.2261 - val_loss: 1.9426 - val_accuracy: 0.2991
Epoch 3/30
532/532 [==============================] - 155s 291ms/step - loss: 1.9573 - accuracy: 0.2790 - val_loss: 1.8712 - val_accuracy: 0.3298
Epoch 4/30
532/532 [==============================] - 153s 287ms/step - loss: 1.8813 - accuracy: 0.3192 - val_loss: 1.7944 - val_accuracy: 0.3627
Epoch 5/30
532/532 [==============================] - 151s 284ms/step - loss: 1.8242 - accuracy: 0.3398 - val_loss: 1.7505 - val_accuracy: 0.3627
Epoch 6/30
532/532 [==============================] - 152s 286ms/step - loss: 1.7825 - accuracy: 0.3525 - val_loss: 1.7113 - val_accuracy: 0.3744
Epoch 7/30
129/532 [======>.......................] - ETA: 1:48 - loss: 1.7580 - accuracy: 0.3674